# Initialize

In [1]:
TODO https://panel.holoviz.org/user_guide/Pipelines.html

# Options

## Debugging level
debug_level = 0

## Anonymize toggle
anonymize = False


In [2]:
# Initialize framework
constants, tools = init_preprocessing(debug_level)
%matplotlib inline


In [3]:
# Build the anonymous customer name mapping

## Read the datasets into Pandas DataFrames
atlas               = pd.read_csv(constants.RAW_ATLAS_FILE_PATH, index_col="Customers")
forecast            = pd.read_csv(constants.RAW_FORECAST_DATA_FILE_PATH, index_col="Organization Name")
revenue2020         = pd.read_csv(constants.RAW_REVENUE2020_FILE_PATH, index_col="Name")
revenue2020A        = pd.read_csv(constants.RAW_REVENUE2020A_FILE_PATH, index_col="Payee Name")
potential_customers_file    = pd.read_csv(constants.RAW_POTENTIAL_CUSTOMERS_FILE_PATH, index_col="Customer Name")

## Extract paying customers
atlas_customers_raw             = atlas.index.to_list()
forecast_customers_raw          = forecast.index.to_list()
revenue2020_customers_raw       = revenue2020.index.to_list()
revenue2020A_customers_raw      = revenue2020A.index.to_list()
potential_customers_raw         = potential_customers_file.index.to_list()

## Clean up names
atlas_customers_cleaned             = tools.tool_customer.cleanup_customer_names(atlas_customers_raw)
forecast_customers_cleaned          = tools.tool_customer.cleanup_customer_names(forecast_customers_raw)
revenue2020_customers_cleaned       = tools.tool_customer.cleanup_customer_names(revenue2020_customers_raw)
revenue2020A_customers_cleaned      = tools.tool_customer.cleanup_customer_names(revenue2020A_customers_raw)
potential_customers_cleaned         = tools.tool_customer.cleanup_customer_names(potential_customers_raw)

## Build name mapping
name_mapping                            = tools.tool_mapping.build_name_mapping(atlas_customers_cleaned, {})
name_mapping                            = tools.tool_mapping.build_name_mapping(forecast_customers_cleaned, name_mapping)
name_mapping                            = tools.tool_mapping.build_name_mapping(revenue2020_customers_cleaned, name_mapping)
name_mapping                            = tools.tool_mapping.build_name_mapping(revenue2020A_customers_cleaned, name_mapping)
name_mapping_potential_customers        = tools.tool_mapping.build_name_mapping(potential_customers_cleaned, {})

## Export the name mappings
tools.tool_mapping.write_customer_name_mapping(name_mapping)
tools.tool_mapping.write_lookup_table(name_mapping_potential_customers, constants.LUT_POTENTIAL_CUSTOMER_NAME_FILE_PATH)

## Export the customer name list
if anonymize:
    tools.tool_mapping.write_list( list(set(name_mapping.values())),
        constants.LIST_CUSTOMER_NAME_LIST_FILE_PATH)
    tools.tool_mapping.write_list( list(set(name_mapping_potential_customers.values())),
        constants.LIST_POTENTIAL_CUSTOMER_NAME_LIST_FILE_PATH)
else:
    tools.tool_mapping.write_list( list(set(name_mapping.keys())),
        constants.LIST_CUSTOMER_NAME_LIST_FILE_PATH)
    tools.tool_mapping.write_list( list(set(name_mapping_potential_customers.keys())),
        constants.LIST_POTENTIAL_CUSTOMER_NAME_LIST_FILE_PATH)


In [4]:
# Anonymize the raw data files

## Clean up names
atlas_customers_cleaned         = tools.tool_customer.cleanup_customer_names(atlas_customers_raw)
forecast_customers_cleaned      = tools.tool_customer.cleanup_customer_names(forecast_customers_raw)
revenue2020_customers_cleaned   = tools.tool_customer.cleanup_customer_names(revenue2020_customers_raw)
revenue2020A_customers_cleaned  = tools.tool_customer.cleanup_customer_names(revenue2020A_customers_raw)


if anonymize:

    print("Anonymizing data!")

    ## Import name mapping
    customer_name_mapping = tool_mapping.read_customer_name_mapping()

    ## Map the customer names
    atlas_customers_finalized              = tools.tool_customer.anonymize_customer_list(atlas_customers_cleaned, customer_name_mapping)
    forecast_customers_finalized           = tools.tool_customer.anonymize_customer_list(forecast_customers_cleaned, customer_name_mapping)
    revenue2020_customers_finalized        = tools.tool_customer.anonymize_customer_list(revenue2020_customers_cleaned, customer_name_mapping)
    revenue2020A_customers_finalized       = tools.tool_customer.anonymize_customer_list(revenue2020A_customers_cleaned, customer_name_mapping)


else:
    
    print("NOT anonymizing data!")
    atlas_customers_finalized = atlas_customers_cleaned
    forecast_customers_finalized = forecast_customers_cleaned
    revenue2020_customers_finalized = revenue2020_customers_cleaned
    revenue2020A_customers_finalized = revenue2020A_customers_cleaned


# Fix the customers in the DataFrames
atlas           = tools.tool_customer.set_customers_index(atlas, atlas_customers_finalized, "Customers")
forecast        = tools.tool_customer.set_customers_index(forecast, forecast_customers_finalized, "Organization Name")
revenue2020     = tools.tool_customer.set_customers_index(revenue2020, revenue2020_customers_finalized, "Name")
revenue2020A    = tools.tool_customer.set_customers_index(revenue2020A, revenue2020A_customers_finalized, "Payee Name")

NOT anonymizing data!


In [5]:
# Export
atlas.to_csv(constants.ANON_ATLAS_FILE_PATH)
forecast.to_csv(constants.ANON_FORECAST_DATA_FILE_PATH)
revenue2020.to_csv(constants.ANON_REVENUE2020_FILE_PATH)
revenue2020A.to_csv(constants.ANON_REVENUE2020A_FILE_PATH)
